In [2]:
import datasets
import pandas as pd


aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']

# for aspect in aspects:
#     df1 = pd.read_excel(f'../../chatgpt/outputs/main_data_batch_gold_results.xlsx', sheet_name=aspect)
#     df2 = pd.read_excel(f'../../chatgpt/outputs/main_data_batch_silver_results.xlsx', sheet_name=aspect)

#     chatgpt_df = pd.concat([df1, df2])

#     chatgpt_df = chatgpt_df [chatgpt_df[f'{aspect}_label'] == chatgpt_df[f'chatgpt_{aspect}_score']]

#     print(aspect, len(chatgpt_df['id'].unique()))


#     for x in chatgpt_df[f'chatgpt_{aspect}_rationale'].head(5):
#         print(x)


In [5]:
test_set = datasets.load_dataset("boda/review_evaluation_automatic_labels", name= "all", split="test")
test_set = test_set.to_pandas()

In [7]:


sampled_test_set = test_set.sample(n=100, random_state=42)

In [8]:
for aspect in aspects:
    sampled_test_set[f"{aspect}_rationale_correctness"] = None
    sampled_test_set[f"{aspect}_rationale_relevance"] = None


sampled_test_set.to_csv('human_rationale_evaluation.csv', index=False)